In [38]:
# import of the needed packages
import pandas as pd
import numpy as np
import re
import math
import csv

file = 'cumulated_data_bl1_v3_with_mw_and_form.csv'

In [39]:
def load_data(file):
    data = pd.read_csv(file, delimiter=',')
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0
    return data

In [40]:
def get_all_teams(data):
    teams = []
    for x in data['HomeTeam']:
        if x not in teams:
            teams.append(x)
    return teams

In [41]:
def create_elo_dictionary(teams):
    teams_elo_avstaerke = {}
    for x in teams:
        team_stats = {x: [1000, 1000, 1.34, -1.34]}
        teams_elo_avstaerke.update(team_stats)
    return teams_elo_avstaerke

In [42]:
def diffelo(homeelo, awayelo):
    diffelo = homeelo - awayelo
    return diffelo

In [43]:
def elogain_2(z, data, hometeam, awayteam, teams_elo_avstaerke):
    homeelo = teams_elo_avstaerke[hometeam][0]
    awayelo = teams_elo_avstaerke[awayteam][0]

    trueoutcome = outcometrue_2(z, data)
    expectedoutcome = outcomeexpected_2(z, homeelo, awayelo)
    ofvictorymargin = marginofvictory_2(z, data, homeelo, awayelo)

    elogain = 20 * (trueoutcome - expectedoutcome) * ofvictorymargin
    elohome = homeelo + elogain
    eloaway = awayelo - elogain

    teams_elo_avstaerke[hometeam][0] = elohome
    teams_elo_avstaerke[awayteam][1] = eloaway

    return elohome, eloaway, homeelo, awayelo

In [44]:
def marginofvictory_2(z, data, homeelo, awayelo):
    elodiff = diffelo(homeelo, awayelo)
    diffgoal = data.FTHG[z] - data.FTAG[z]

    if diffgoal <= 1:
        marginofvictory = 1
    else:
        marginofvictory = (math.log2(1.7 * diffgoal) * 2) / (2 +
                                                             0.001 * elodiff)
    return marginofvictory

In [45]:
def outcometrue_2(z, data):
    if data.FTR[z] == "H":
        outcometrue = 1
    elif data.FTR[z] == "D":
        outcometrue = 0.5
    else:
        outcometrue = 0
    return outcometrue

In [46]:
def outcomeexpected_2(z, homeelo, awayelo):
    elodiff = diffelo(homeelo, awayelo)
    outcomeexpected = 1 / (1 + 10**(-elodiff / 400))
    return outcomeexpected

In [47]:
def home_attack(z, data, alpha, ratio, hometeam, awayteam,
                teams_elo_avstaerke):
    goalsshot = data.FTHG[z]
    attack_old = teams_elo_avstaerke[hometeam][2]
    otherteamdefend = teams_elo_avstaerke[awayteam][3]
    
    homeattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[hometeam][2] = homeattack
    return homeattack, attack_old

In [48]:
def home_defend(z, alpha, ratio, data, hometeam, awayteam,
                teams_elo_avstaerke):
    goalsgot = data.FTAG[z]
    defend_old = teams_elo_avstaerke[hometeam][3]
    otherteamattack = teams_elo_avstaerke[awayteam][2]
    
    homedefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[hometeam][3] = homedefend
    return homedefend, defend_old

In [49]:
def away_attack(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    goalsshot = data.FTAG[z]
    attack_old = teams_elo_avstaerke[awayteam][2]
    otherteamdefend = teams_elo_avstaerke[hometeam][3]
    
    awayattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[awayteam][2] = awayattack
    return awayattack, attack_old

In [50]:
def away_defend(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    goalsgot = data.FTHG[z]
    defend_old = teams_elo_avstaerke[awayteam][3]
    otherteamattack = teams_elo_avstaerke[hometeam][2]
    awaydefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    teams_elo_avstaerke[awayteam][3] = awaydefend
    return awaydefend, defend_old

In [51]:
def add_new_columns1(data):
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0

In [52]:
def add_new_columns2(data):
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0

In [53]:
def calculate_elo_attack_defense(data, teams_elo_avstaerke):
    z = 0
    while z < len(data):
        alpha = 0.5
        ratio = 0.75
        hometeam = data.HomeTeam[z]
        awayteam = data.AwayTeam[z]

        try:

            homeelo, awayelo, homeeloold, awayeloold = elogain_2(
            z, data, hometeam, awayteam, teams_elo_avstaerke)
            homeattack1, homeattackold = home_attack(z, data, alpha, ratio,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            homedefend1, homedefendold = home_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            awayattack1, awayattackold = away_attack(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            awaydefend1, awaydefendold = away_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
        except:
            print(f"Can't find data for team: {hometeam, awayteam}")
            break
        data['HomeElo'][z] = homeelo
        data['AwayElo'][z] = awayelo
        data['HomeAttack'][z] = homeattack1
        data['HomeDefend'][z] = homedefend1
        data['AwayAttack'][z] = awayattack1
        data['AwayDefend'][z] = awaydefend1
        data['HomeEloOld'][z] = homeeloold
        data['AwayEloOld'][z] = awayeloold
        data['HomeAttackOld'][z] = homeattackold
        data['HomeDefendOld'][z] = homedefendold
        data['AwayAttackOld'][z] = awayattackold
        data['AwayDefendOld'][z] = awaydefendold
        z+=1
    return data
    # return homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold


In [54]:
def add_elo_attack_defense(homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold):
    data['HomeElo'][z] = homeelo
    data['AwayElo'][z] = awayelo
    data['HomeAttack'][z] = homeattack1
    data['HomeDefend'][z] = homedefend1
    data['AwayAttack'][z] = awayattack1
    data['AwayDefend'][z] = awaydefend1
    data['HomeEloOld'][z] = homeeloold
    data['AwayEloOld'][z] = awayeloold
    data['HomeAttackOld'][z] = homeattackold
    data['HomeDefendOld'][z] = homedefendold
    data['AwayAttackOld'][z] = awayattackold
    data['AwayDefendOld'][z] = awaydefendold


In [55]:
def add_differences(data):
    data['DiffEloOld'] = data['HomeEloOld'] - data['AwayEloOld']
    data['DiffAttackOld'] = data['HomeAttackOld'] - data['AwayAttackOld']
    data['DiffDefendOld'] = data['HomeDefendOld'] - data['AwayDefendOld']
    data['DiffElo'] = data['HomeElo'] - data['AwayElo']
    data['DiffAttack'] = data['HomeAttack'] - data['AwayAttack']
    data['DiffDefend'] = data['HomeDefend'] - data['AwayDefend']
    return data

In [56]:
def add_qoutient(data):
    data['QuotEloOld'] = (data['HomeEloOld']) / (data['AwayEloOld'])
    data['QuotAttackOld'] = (data['HomeAttackOld']) / (data['AwayAttackOld'])
    data['QuotDefendOld'] = (data['HomeDefendOld']) / (data['AwayDefendOld'])
    data['QuotElo'] = (data['HomeElo']) / (data['AwayElo'])
    data['QuotAttack'] = (data['HomeAttack']) / (data['AwayAttack'])
    data['QuotDefend'] = (data['HomeDefend']) / (data['AwayDefend'])
    return data

In [57]:
def direct_comparison(data):
    # get direct comparison of the two teams

    direct_comparison_hg = []
    direct_comparison_ag = []
    direct_comparison_hp = []
    direct_comparison_ap = []

    for counter, rows in data.iterrows():
        data_games_played_until_now = data.iloc[:counter]
        direct_comparison_teams_temp = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)&(data_games_played_until_now.AwayTeam == rows.AwayTeam)|
        (data_games_played_until_now.HomeTeam == rows.AwayTeam)&(data_games_played_until_now.AwayTeam == rows.HomeTeam)]
        if len(direct_comparison_teams_temp) >= 3:
            if len(direct_comparison_teams_temp) >= 5:
                # take only the last 5 matches
                direct_comparison_teams_temp = direct_comparison_teams_temp.iloc[-5:]
                #direct_comparison_teams_temp.iloc[-5:].value_counts(["HomeTeam", "FTAG"])
                number_of_games = 5
            else:
                number_of_games = len(direct_comparison_teams_temp)

            # get average number of goals, that the hometeam has scored in the last 5 direct matches
            home_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.HomeTeam].FTAG.sum()) / number_of_games

            # get average number of goals, that the awayteam has scored in the last 5 direct matches
            away_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.AwayTeam].FTAG.sum()) / number_of_games

        else:
            # if the direct comparison is not possible yet (because both teams haven't played against each other) the scored goals of the last 10 matches would be measured to avoid null values
            goals_scored_ht = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)|(data_games_played_until_now.AwayTeam == rows.HomeTeam)].iloc[-10:]
            goals_scored_at = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.AwayTeam)|(data_games_played_until_now.AwayTeam == rows.AwayTeam)].iloc[-10:]

            if len(goals_scored_ht) >= 10:
                home_goals = (goals_scored_ht[goals_scored_ht.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                    goals_scored_ht[goals_scored_ht.AwayTeam == rows.HomeTeam].FTAG.sum()) / 10
                away_goals = (goals_scored_at[goals_scored_at.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                    goals_scored_at[goals_scored_at.AwayTeam == rows.AwayTeam].FTAG.sum()) / 10
            else:
                home_goals = None
                away_goals = None

        direct_comparison_hg.append(home_goals)
        direct_comparison_ag.append(away_goals)

    # print(direct_comparison_hg)
    # print(direct_comparison_ag)
    # print(direct_comparison_hp)
    # print(direct_comparison_ap)

    data['DirectComparisonHG'] = direct_comparison_hg
    data['DirectComparisonAG'] = direct_comparison_ag
    data['DirectComparisonGoalDiff'] = data[
            'DirectComparisonHG'] - data['DirectComparisonAG']
    data['DirectComparisonGoalQuot'] = (data['DirectComparisonHG']) / (data[
    'DirectComparisonAG']+1)
    return data


In [58]:
def select_necessary_columns(data):
    data_reduced = data[[
        "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "HS", "AS",
        "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "PointsLast3MatchesHT",
        "PointsLast3MatchesAT", "PointsLast5MatchesHT", "PointsLast5MatchesAT",
        "PointsLast10MatchesHT", "PointsLast10MatchesAT", "PointsLastAllMatchesHT",
        "PointsLastAllMatchesAT", "PDiff3Matches", "PQuot3Matches", "PDiff5Matches",
        "PQuot5Matches", "PDiff10Matches", "PQuot10Matches", "PDiffAllMatches",
        "PQuotAllMatches", "MarketValueHT", "MarketValueAT", "MarketValueDiff",
        "MarketValueQuot", "HomeElo", "AwayElo", "HomeAttack", "HomeDefend", "AwayAttack",
        "AwayDefend", "HomeEloOld", "AwayEloOld", "HomeAttackOld", "HomeDefendOld",
        "AwayAttackOld", "AwayDefendOld", "DiffEloOld", "DiffAttackOld", "DiffDefendOld",
        "DiffElo", "DiffAttack", "DiffDefend", "QuotEloOld", "QuotAttackOld",
        "QuotDefendOld", "QuotElo", "QuotAttack", "QuotDefend", "DirectComparisonHG",
        "DirectComparisonAG", "DirectComparisonGoalDiff", "DirectComparisonGoalQuot"
    ]]
    return data_reduced


In [59]:
def save_to_csv(data_reduced):
    data_reduced.to_csv("./Data/preprocessed_dataframe_with_elo_mw_form_3_v3.csv")

In [60]:
def run_all(file):
    data = load_data(file)
    teams = get_all_teams(data)
    teams_elo_avstaerke = create_elo_dictionary(teams)
    data = calculate_elo_attack_defense(data, teams_elo_avstaerke)
    data = add_differences(data)
    data = add_qoutient(data)
    data = direct_comparison(data)
    data_reduced = select_necessary_columns(data)
    save_to_csv(data_reduced)
    return data_reduced


data = run_all(file)

C:\Users\I539022\AppData\Local\Temp/ipykernel_25056/127603076.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HomeElo'][z] = homeelo
C:\Users\I539022\AppData\Local\Temp/ipykernel_25056/127603076.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['AwayElo'][z] = awayelo
C:\Users\I539022\AppData\Local\Temp/ipykernel_25056/127603076.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HomeAttack'][z] = homeattack1
